<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EC%9C%A0%EC%98%88%EC%8A%AC_%EB%A9%80%ED%8B%B0%EB%9D%BC%EB%B2%A8_%ED%95%99%EC%8A%B5_5_categories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [304]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [305]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/멀티캠퍼스 자료/Machine Learning/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# 네이버 데이터

In [306]:
nv = pd.read_excel(f"{DATA_PATH}naver_reviews_19581_data.xlsx")
nv.to_csv(f"{DATA_PATH}naver_reviews_1.csv")

In [307]:
naver = pd.read_csv(f"{DATA_PATH}naver_reviews_1.csv")

In [308]:
naver.isnull().sum(), naver.shape

(Unnamed: 0       0
 id               0
 review         463
 date             0
 date_cnt         0
 keyword       1636
 dtype: int64,
 (19580, 6))

In [309]:
naver.info() # 총 19580개 데이터, review에 463 결측치, 키워드 1636 결측치

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19580 entries, 0 to 19579
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  19580 non-null  int64 
 1   id          19580 non-null  object
 2   review      19117 non-null  object
 3   date        19580 non-null  object
 4   date_cnt    19580 non-null  int64 
 5   keyword     17944 non-null  object
dtypes: int64(2), object(4)
memory usage: 917.9+ KB


In [310]:
df = naver.copy()

In [311]:
df.head(10)

,Unnamed: 0,id,review,date,date_cnt,keyword
0,0,namju677,마드레~생산구이정식 맛있고 반찬 많아서 \n외식메뉴로 좋아요.~~,8.5.토,1,"음식이 맛있어요,양이 많아요"
1,1,탱글이136,정말 맛있어요~! 반찬도 다양해서 너무 좋아요!!\n최고👍특히 생선구이가 제일인것 ...,6.19.월,1,"음식이 맛있어요,양이 많아요,가성비가 좋아요,친절해요"
2,2,큐웅49,부모님 모시고 오기 딱 좋은 음식점이예요. 반찬부터 생선구이 모든게 다 맛있습니다.,7.1.토,1,"음식이 맛있어요,인테리어가 멋져요"
3,3,오잉hj,"맛은 쏘쏘인데, 자반정식 2인분 3만2000원인데 한마리(1인분에 반마리인 셈) 나...",6.30.금,1,"주차하기 편해요,음식이 맛있어요,인테리어가 멋져요"
4,4,safetykts,예쁜 꽃이 반겨주고 맛난밥이 기분좋게해주는~ 근데 어버이날 특수여서? 간장게장을 ...,5.9.화,1,"특별한 날 가기 좋아요,친절해요,매장이 청결해요,뷰가 좋아요,차분한 분위기에요"
5,5,Joshhart,"근처에 갈 일이 있어 저녁을 해결할 곳을 찾다 발견했는데, 리뷰에서 호불호가 느껴졌...",7.9.일,1,"음식이 맛있어요,건강한 맛이에요,반찬이 잘 나와요,단체모임 하기 좋아요,차분한 분위기에요"
6,6,동글히,주차할 때 공작새가 맘껏 뽐내고 있는 자태에 놀라고 밥 먹으러 들어가는 길에 기러기...,5.30.화,1,"음식이 맛있어요,단체모임 하기 좋아요"
7,7,니콜라1215,어른들 모시고 갔는데 밑반찬과 갈비찜이 맛있었어요. 황태구이는 기름져서 별로 였지만...,5.15.월,1,"양이 많아요,음식이 맛있어요,주차하기 편해요"
8,8,ban****,맛있어요 식사가 정성을 많이들인거 같아요,7.23.일,1,"음식이 맛있어요,재료가 신선해요,특별한 메뉴가 있어요,가성비가 좋아요,매장이 넓어요"
9,9,행복한도로시는나야,예전만 못 하네,8.9.수,1,NaN


In [312]:
# NaN 값이 있는 행 제거
df.dropna(subset=['keyword', 'review'], inplace=True)

# 빈 문자열인 행 제거
df = df[df['review'].str.strip() != ""]

# 공백 제거
df['review'] = df['review'].str.strip()

# 라인 끝의 '\n' 제거
df['review'] = df['review'].str.replace('\n', ' ')

# 한글과 숫자를 제외한 모든 문자 제거
pattern = r'[^0-9가-힣\s]'
df['review'] = df['review'].str.replace(pattern, '', regex=True)

# 'keyword'와 'review' 열이 공백인 행을 제거
df = df[(df['keyword'] != '') & (df['review'] != '')]

In [313]:
df.isnull().sum()

Unnamed: 0    0
id            0
review        0
date          0
date_cnt      0
keyword       0
dtype: int64

In [314]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17342 entries, 0 to 19579
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  17342 non-null  int64 
 1   id          17342 non-null  object
 2   review      17342 non-null  object
 3   date        17342 non-null  object
 4   date_cnt    17342 non-null  int64 
 5   keyword     17342 non-null  object
dtypes: int64(2), object(4)
memory usage: 948.4+ KB


In [315]:
import random

random_samples = df.sample(n=5)

for index, row in random_samples.iterrows():
    print(f"Review: {row['review']}\nKeywords: {row['keyword']}\n---")


Review: 좋아요
Keywords: 음식이 맛있어요
---
Review: 음식이 맛잇어요
Keywords: 매장이 청결해요,친절해요,재료가 신선해요,음식이 맛있어요
---
Review: 다른건 다 맛있었는데 흑임자파스타는 사진을 못찍었는데 너무 짰어요 대체적으로 엄청 맛있다정도 까지는 아니고 조화로우면서 좋은재료 쓴 느낌이였어요 육회비빔밥이랑 항정살 맛있습니다 명란비빔밥은 슴슴해요
Keywords: 재료가 신선해요,음식이 맛있어요,매장이 청결해요
---
Review: 밥이맛있어요
Keywords: 음식이 맛있어요, 재료가 신선해요, 양이 많아요, 가성비가 좋아요, 매장이 넓어요
---
Review: 딱새우 크으 연어도 맛있고 라면도 맛있고 하이볼 넘 맛있었어요 서비스도 주셔서 배불리 먹고 갔으용
Keywords: 술이 다양해요, 음식이 맛있어요, 특별한 메뉴가 있어요
---


In [316]:
df.isnull().sum()

Unnamed: 0    0
id            0
review        0
date          0
date_cnt      0
keyword       0
dtype: int64

# 네이버 라벨 확인
- 총 52개의 라벨


In [317]:
total_label = []
for label in df['keyword']:
    # Use split(',') to split the string into a list of substrings
    labels_list = label.split(',')
    # Remove leading and trailing whitespace from each substring
    cleaned_labels = [item.strip() for item in labels_list]

    # Extend the all_label list with the cleaned_labels
    total_label.extend(cleaned_labels)

In [318]:
total_label

['음식이 맛있어요',
 '양이 많아요',
 '음식이 맛있어요',
 '양이 많아요',
 '가성비가 좋아요',
 '친절해요',
 '음식이 맛있어요',
 '인테리어가 멋져요',
 '주차하기 편해요',
 '음식이 맛있어요',
 '인테리어가 멋져요',
 '특별한 날 가기 좋아요',
 '친절해요',
 '매장이 청결해요',
 '뷰가 좋아요',
 '차분한 분위기에요',
 '음식이 맛있어요',
 '건강한 맛이에요',
 '반찬이 잘 나와요',
 '단체모임 하기 좋아요',
 '차분한 분위기에요',
 '음식이 맛있어요',
 '단체모임 하기 좋아요',
 '양이 많아요',
 '음식이 맛있어요',
 '주차하기 편해요',
 '음식이 맛있어요',
 '재료가 신선해요',
 '특별한 메뉴가 있어요',
 '가성비가 좋아요',
 '매장이 넓어요',
 '음식이 맛있어요',
 '친절해요',
 '뷰가 좋아요',
 '음식이 맛있어요',
 '주차하기 편해요',
 '특별한 날 가기 좋아요',
 '음식이 맛있어요',
 '친절해요',
 '음식이 맛있어요',
 '음식이 맛있어요',
 '재료가 신선해요',
 '양이 많아요',
 '특별한 메뉴가 있어요',
 '가성비가 좋아요',
 '단체모임 하기 좋아요',
 '음식이 맛있어요',
 '주차하기 편해요',
 '인테리어가 멋져요',
 '대화하기 좋아요',
 '컨셉이 독특해요',
 '반찬이 잘 나와요',
 '매장이 넓어요',
 '친절해요',
 '주차하기 편해요',
 '특별한 날 가기 좋아요',
 '음식이 맛있어요',
 '재료가 신선해요',
 '양이 많아요',
 '주차하기 편해요',
 '음식이 맛있어요',
 '뷰가 좋아요',
 '단체모임 하기 좋아요',
 '주차하기 편해요',
 '가성비가 좋아요',
 '음식이 맛있어요',
 '친절해요',
 '주차하기 편해요',
 '음식이 맛있어요',
 '특별한 메뉴가 있어요',
 '음식이 맛있어요',
 '양이 많아요',
 '음식이 맛있어요',
 '특별한 메뉴가 있어요',
 '단체모임 하기 좋아요',
 '뷰가 좋아요',
 '음식이 맛있어요',

In [319]:
label_type = set(total_label)
label_type = list(label_type)
len(label_type)

52

## 라벨 유형 확인 (카테고리)

- 총 5개 유형으로 분류 : 맛, 양, 가격, 공간, 서비스

In [320]:
taste_keywords = ['맛','반찬','사이드','안주','건강', '향신료', '재료', '잡내', '질']
quantity_keywords = ['양','알차']
price_keywords = ['가격', '가성비', '비싼', '가치']
place_keywords = ['공간', '아늑', '조용', '분위기', '모임', '혼밥','혼술', '청결', '환기', '깨끗', '컨셉', '대화', '주차', '룸', '집중', '음악', '사진', '날', '좌석', '아이', '뷰', '인테리어', '매장', '반려동물', '화장실', '바', '특별', '뷰','머무르']

taste_label = []
quantity_label = []
price_label = []
place_label = []
service_label = []

for label in label_type:
    found = False
    for keyword in taste_keywords:
        if keyword in label:
            taste_label.append(label)
            found = True
            break
    if not found:
        for keyword in quantity_keywords:
            if keyword in label:
                quantity_label.append(label)
                found = True
                break
    if not found:
        for keyword in price_keywords:
            if keyword in label:
                price_label.append(label)
                found = True
                break
    if not found:
        for keyword in place_keywords:
            if keyword in label:
                place_label.append(label)
                found = True
                break
    if not found:
        service_label.append(label)

In [321]:
taste_label     #기본안주, 반찬 칭찬은 맛 관련으로 가정

['빵이 맛있어요',
 '디저트가 맛있어요',
 '건강한 맛이에요',
 '잡내가 적어요',
 '현지 맛에 가까워요',
 '차가 맛있어요',
 '반찬이 잘 나와요',
 '재료가 신선해요',
 '커피가 맛있어요',
 '향신료가 강하지 않아요',
 '기본 안주가 좋아요',
 '음료가 맛있어요',
 '음식이 맛있어요',
 '고기 질이 좋아요']

In [322]:
quantity_label

['종류가 다양해요', '술이 다양해요', '코스요리가 알차요', '양이 많아요', '메뉴 구성이 알차요']

In [323]:
price_label

['가성비가 좋아요', '비싼 만큼 가치있어요']

In [324]:
place_label

['오래 머무르기 좋아요',
 '컨셉이 독특해요',
 '환기가 잘 돼요',
 '좌석이 편해요',
 '화장실이 깨끗해요',
 '차분한 분위기에요',
 '단체모임 하기 좋아요',
 '반려동물과 가기 좋아요',
 '특별한 메뉴가 있어요',
 '인테리어가 멋져요',
 '혼밥하기 좋아요',
 '뷰가 좋아요',
 '혼술하기 좋아요',
 '집중하기 좋아요',
 '매장이 넓어요',
 '대화하기 좋아요',
 '야외 공간이 멋져요',
 '아늑해요',
 '매장이 청결해요',
 '음악이 좋아요',
 '샐러드바가 잘 되어있어요',
 '주차하기 편해요',
 '특별한 날 가기 좋아요',
 '룸이 잘 되어있어요',
 '아이와 가기 좋아요',
 '사진이 잘 나와요']

In [325]:
service_label

['직접 잘 구워줘요', '친절해요', '선물하기 좋아요', '포장이 깔끔해요', '음식이 빨리 나와요']

## 라벨 분포

In [326]:
from collections import Counter

# all_keywords 리스트에 중복된 키워드의 개수를 세기
keyword_counter = Counter(total_label)

# 빈도수가 높은 순으로 키워드 출력
for keyword, count in keyword_counter.most_common():
    print(f"{keyword}: {count}")


음식이 맛있어요: 12110
친절해요: 6204
재료가 신선해요: 4724
특별한 메뉴가 있어요: 4511
매장이 청결해요: 3911
인테리어가 멋져요: 3414
가성비가 좋아요: 3341
양이 많아요: 3075
매장이 넓어요: 2887
혼밥하기 좋아요: 1824
디저트가 맛있어요: 1321
주차하기 편해요: 1290
특별한 날 가기 좋아요: 1196
단체모임 하기 좋아요: 911
뷰가 좋아요: 874
커피가 맛있어요: 715
대화하기 좋아요: 489
음료가 맛있어요: 484
사진이 잘 나와요: 444
화장실이 깨끗해요: 421
고기 질이 좋아요: 327
좌석이 편해요: 260
메뉴 구성이 알차요: 236
음식이 빨리 나와요: 184
빵이 맛있어요: 147
현지 맛에 가까워요: 146
포장이 깔끔해요: 145
차분한 분위기에요: 101
아늑해요: 100
비싼 만큼 가치있어요: 98
음악이 좋아요: 98
집중하기 좋아요: 91
컨셉이 독특해요: 89
술이 다양해요: 75
건강한 맛이에요: 73
기본 안주가 좋아요: 71
직접 잘 구워줘요: 65
오래 머무르기 좋아요: 50
아이와 가기 좋아요: 43
야외 공간이 멋져요: 26
반찬이 잘 나와요: 25
선물하기 좋아요: 21
종류가 다양해요: 19
혼술하기 좋아요: 13
향신료가 강하지 않아요: 12
잡내가 적어요: 10
룸이 잘 되어있어요: 7
반려동물과 가기 좋아요: 3
코스요리가 알차요: 3
샐러드바가 잘 되어있어요: 3
차가 맛있어요: 2
환기가 잘 돼요: 1


## 카테고리 분포
- 맛 35%, 공간 40% (지우거나 배달로 대체 필요)
- 가격 6%, 양 6%, 서비스 12%

In [327]:
categories = ['taste_label','quantity_label', 'price_label', 'place_label','service_label' ]

labels_by_category = {
    'taste_label' : taste_label,
    'quantity_label' : quantity_label,
    'price_label' : price_label,
    'place_label' : place_label,
    'service_label' : service_label
}

In [328]:
for category in categories :
    for label in total_label:
        if label in labels_by_category[category]:
            count += 1

    print(f'{category}: {count}')

taste_label: 20168
quantity_label: 23576
price_label: 27015
place_label: 50072
service_label: 56691


In [329]:
import pandas as pd

# Create an empty DataFrame
category_counts_df = pd.DataFrame()

category_counts = {}

for category in categories:
    count = 0  # Initialize count for each category
    for label in total_label:
        if label in labels_by_category[category]:
            count += 1  # Increment the count for the current category
    category_counts[category] = count

category_counts_df['Category'] = category_counts.keys()
category_counts_df['Count'] = category_counts.values()

total_count = len(total_label)
category_counts_df['Count Proportion'] = category_counts_df['Count'] / total_count

# Display the DataFrame
category_counts_df



,Category,Count,Count Proportion
0,taste_label,20167,0.355742
1,quantity_label,3408,0.060116
2,price_label,3439,0.060663
3,place_label,23057,0.406721
4,service_label,6619,0.116758


# 데이터를 카테고리 기준으로 재분류

In [330]:
train = df[['review','keyword']]
train.columns = ['review','labels']

train

,review,labels
0,마드레생산구이정식 맛있고 반찬 많아서 외식메뉴로 좋아요,"음식이 맛있어요,양이 많아요"
1,정말 맛있어요 반찬도 다양해서 너무 좋아요 최고특히 생선구이가 제일인것 같아요 불고...,"음식이 맛있어요,양이 많아요,가성비가 좋아요,친절해요"
2,부모님 모시고 오기 딱 좋은 음식점이예요 반찬부터 생선구이 모든게 다 맛있습니다,"음식이 맛있어요,인테리어가 멋져요"
3,맛은 쏘쏘인데 자반정식 2인분 3만2000원인데 한마리1인분에 반마리인 셈 나오는건 좀,"주차하기 편해요,음식이 맛있어요,인테리어가 멋져요"
4,예쁜 꽃이 반겨주고 맛난밥이 기분좋게해주는 근데 어버이날 특수여서 간장게장을 먹을...,"특별한 날 가기 좋아요,친절해요,매장이 청결해요,뷰가 좋아요,차분한 분위기에요"
...,...,...
19575,아주아주 맛있습니다 동백카츠 최고,"재료가 신선해요,음악이 좋아요,친절해요"
19576,깔끔하고 맛있어요,"음식이 맛있어요,인테리어가 멋져요"
19577,첫 방문 항상 사람 많아 못 들어왔는데 오늘은 대기없이 바로 들어올수 있어 굿 맛 ...,"친절해요,매장이 청결해요,재료가 신선해요,특별한 메뉴가 있어요,음식이 맛있어요"
19578,직원분들이 친절해요,"인테리어가 멋져요,매장이 넓어요,음식이 맛있어요,재료가 신선해요"


In [331]:
train['category'] = None
train

<ipython-input-331-dad341f576b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['category'] = None


,review,labels,category
0,마드레생산구이정식 맛있고 반찬 많아서 외식메뉴로 좋아요,"음식이 맛있어요,양이 많아요",None
1,정말 맛있어요 반찬도 다양해서 너무 좋아요 최고특히 생선구이가 제일인것 같아요 불고...,"음식이 맛있어요,양이 많아요,가성비가 좋아요,친절해요",None
2,부모님 모시고 오기 딱 좋은 음식점이예요 반찬부터 생선구이 모든게 다 맛있습니다,"음식이 맛있어요,인테리어가 멋져요",None
3,맛은 쏘쏘인데 자반정식 2인분 3만2000원인데 한마리1인분에 반마리인 셈 나오는건 좀,"주차하기 편해요,음식이 맛있어요,인테리어가 멋져요",None
4,예쁜 꽃이 반겨주고 맛난밥이 기분좋게해주는 근데 어버이날 특수여서 간장게장을 먹을...,"특별한 날 가기 좋아요,친절해요,매장이 청결해요,뷰가 좋아요,차분한 분위기에요",None
...,...,...,...
19575,아주아주 맛있습니다 동백카츠 최고,"재료가 신선해요,음악이 좋아요,친절해요",None
19576,깔끔하고 맛있어요,"음식이 맛있어요,인테리어가 멋져요",None
19577,첫 방문 항상 사람 많아 못 들어왔는데 오늘은 대기없이 바로 들어올수 있어 굿 맛 ...,"친절해요,매장이 청결해요,재료가 신선해요,특별한 메뉴가 있어요,음식이 맛있어요",None
19578,직원분들이 친절해요,"인테리어가 멋져요,매장이 넓어요,음식이 맛있어요,재료가 신선해요",None


In [332]:
train['labels'] = train['labels'].apply(lambda x: [k.strip() for k in x.split(',')])

<ipython-input-332-5975f4575784>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['labels'] = train['labels'].apply(lambda x: [k.strip() for k in x.split(',')])


In [333]:
def get_categories(label_list):
    get_categories = []
    for label in label_list:
        for category in categories:
            if label in labels_by_category[category]:
                get_categories.append(category)
                break  # Once a category is assigned, no need to check other categories
    return get_categories

# Apply the function to each row in the DataFrame
train['category'] = train['labels'].apply(get_categories)



<ipython-input-333-f8bb3b7d058e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['category'] = train['labels'].apply(get_categories)


- 카테고리 칼럼을 만들어서 카테고리 멀티라벨을 추가함

In [334]:
train

,review,labels,category
0,마드레생산구이정식 맛있고 반찬 많아서 외식메뉴로 좋아요,"[음식이 맛있어요, 양이 많아요]","[taste_label, quantity_label]"
1,정말 맛있어요 반찬도 다양해서 너무 좋아요 최고특히 생선구이가 제일인것 같아요 불고...,"[음식이 맛있어요, 양이 많아요, 가성비가 좋아요, 친절해요]","[taste_label, quantity_label, price_label, ser..."
2,부모님 모시고 오기 딱 좋은 음식점이예요 반찬부터 생선구이 모든게 다 맛있습니다,"[음식이 맛있어요, 인테리어가 멋져요]","[taste_label, place_label]"
3,맛은 쏘쏘인데 자반정식 2인분 3만2000원인데 한마리1인분에 반마리인 셈 나오는건 좀,"[주차하기 편해요, 음식이 맛있어요, 인테리어가 멋져요]","[place_label, taste_label, place_label]"
4,예쁜 꽃이 반겨주고 맛난밥이 기분좋게해주는 근데 어버이날 특수여서 간장게장을 먹을...,"[특별한 날 가기 좋아요, 친절해요, 매장이 청결해요, 뷰가 좋아요, 차분한 분위기에요]","[place_label, service_label, place_label, plac..."
...,...,...,...
19575,아주아주 맛있습니다 동백카츠 최고,"[재료가 신선해요, 음악이 좋아요, 친절해요]","[taste_label, place_label, service_label]"
19576,깔끔하고 맛있어요,"[음식이 맛있어요, 인테리어가 멋져요]","[taste_label, place_label]"
19577,첫 방문 항상 사람 많아 못 들어왔는데 오늘은 대기없이 바로 들어올수 있어 굿 맛 ...,"[친절해요, 매장이 청결해요, 재료가 신선해요, 특별한 메뉴가 있어요, 음식이 맛있어요]","[service_label, place_label, taste_label, plac..."
19578,직원분들이 친절해요,"[인테리어가 멋져요, 매장이 넓어요, 음식이 맛있어요, 재료가 신선해요]","[place_label, place_label, taste_label, taste_..."


# Multi Label 모델학습***



- 라벨 이름 변경

In [335]:
# 라벨을 한국어로 바꾸기 위한 매핑 딕셔너리 - 모델이 한국어 라벨 학습 더 잘하지 않을지?
label_mapping = {
    'taste_label': '맛',
    'quantity_label' : '양',
    'price_label': '가격',
    'place_label': '장소',
    'service_label': '서비스',
}

# train['category'] = train['category'].replace(label_mapping)

In [336]:
def change_labels(label_list):
    return [label_mapping.get(label, label) for label in label_list]

train['category'] = train['category'].apply(change_labels)

<ipython-input-336-49657dee1524>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['category'] = train['category'].apply(change_labels)


In [337]:
train

,review,labels,category
0,마드레생산구이정식 맛있고 반찬 많아서 외식메뉴로 좋아요,"[음식이 맛있어요, 양이 많아요]","[맛, 양]"
1,정말 맛있어요 반찬도 다양해서 너무 좋아요 최고특히 생선구이가 제일인것 같아요 불고...,"[음식이 맛있어요, 양이 많아요, 가성비가 좋아요, 친절해요]","[맛, 양, 가격, 서비스]"
2,부모님 모시고 오기 딱 좋은 음식점이예요 반찬부터 생선구이 모든게 다 맛있습니다,"[음식이 맛있어요, 인테리어가 멋져요]","[맛, 장소]"
3,맛은 쏘쏘인데 자반정식 2인분 3만2000원인데 한마리1인분에 반마리인 셈 나오는건 좀,"[주차하기 편해요, 음식이 맛있어요, 인테리어가 멋져요]","[장소, 맛, 장소]"
4,예쁜 꽃이 반겨주고 맛난밥이 기분좋게해주는 근데 어버이날 특수여서 간장게장을 먹을...,"[특별한 날 가기 좋아요, 친절해요, 매장이 청결해요, 뷰가 좋아요, 차분한 분위기에요]","[장소, 서비스, 장소, 장소, 장소]"
...,...,...,...
19575,아주아주 맛있습니다 동백카츠 최고,"[재료가 신선해요, 음악이 좋아요, 친절해요]","[맛, 장소, 서비스]"
19576,깔끔하고 맛있어요,"[음식이 맛있어요, 인테리어가 멋져요]","[맛, 장소]"
19577,첫 방문 항상 사람 많아 못 들어왔는데 오늘은 대기없이 바로 들어올수 있어 굿 맛 ...,"[친절해요, 매장이 청결해요, 재료가 신선해요, 특별한 메뉴가 있어요, 음식이 맛있어요]","[서비스, 장소, 맛, 장소, 맛]"
19578,직원분들이 친절해요,"[인테리어가 멋져요, 매장이 넓어요, 음식이 맛있어요, 재료가 신선해요]","[장소, 장소, 맛, 맛]"


## 1) Random Forest

In [338]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Create a RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100)

# Create a MultiOutputClassifier with the RandomForestClassifier
multi_output_classifier = MultiOutputClassifier(rf_classifier)

# Fit the model to the training data (X_train: feature data, y_train: multi-label target data)
multi_output_classifier.fit(X_train, y_train)

# Make predictions on new data
predictions = multi_output_classifier.predict(X_test)


## 2) 사전학습모델 : bongsoo/bert-base-kor-v1
- 자연어 처리 Fill mask 과제 수행 모델 / 문장 분류 작업 가능
- 이 자체로는 multilabel 특화 모델이 아니라 finetuning 필요
- https://huggingface.co/bongsoo/bert-base-kor-v1?text=%ED%95%9C%EA%B5%AD+%EC%88%98%EB%8F%84%EB%8A%94+%5BMASK%5D+%EC%9E%85%EB%8B%88%EB%8B%A4


In [339]:
!pip install transformers

In [340]:
# MultiLabelBinarizer 원핫 인코딩
mlb = MultiLabelBinarizer()
encoded_labels = mlb.fit_transform(train['category'])
label_classes = mlb.classes_

# 토크나이저 및 모델 로딩
from transformers import AutoTokenizer, AutoModel
model_name = "bongsoo/bert-base-kor-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [341]:
# 리뷰 토큰화
train_texts = train['review'].tolist()
train_texts[:10]

['마드레생산구이정식 맛있고 반찬 많아서  외식메뉴로 좋아요',
 '정말 맛있어요 반찬도 다양해서 너무 좋아요 최고특히 생선구이가 제일인것 같아요 불고기는 좀 달았어요',
 '부모님 모시고 오기 딱 좋은 음식점이예요 반찬부터 생선구이 모든게 다 맛있습니다',
 '맛은 쏘쏘인데 자반정식 2인분 3만2000원인데 한마리1인분에 반마리인 셈 나오는건 좀',
 '예쁜 꽃이 반겨주고 맛난밥이 기분좋게해주는  근데 어버이날 특수여서 간장게장을 먹을수가 없었다는그래서  담에 다시가기로',
 '근처에 갈 일이 있어 저녁을 해결할 곳을 찾다 발견했는데 리뷰에서 호불호가 느껴졌지만 간장게장도 먹어보고 싶어서 방문했어요   정식메뉴는 2인분 이상 필수였지만 간장게장 포함 몇몇 메뉴는 1인분 주문이 가능해 불고기정식을 함께 먹었습니다  스텝들이 불친절 하다는 리뷰가 있었으나 전 원래 너무 친절한 건 오히려 불편해서 적당히 좋았어요  반찬들이 싱겁고 달지 않은 건강한 맛에된',
 '주차할 때 공작새가 맘껏 뽐내고 있는 자태에 놀라고 밥 먹으러 들어가는 길에 기러기들 보고 놀랐네요 엄마들 오시면 넘 좋으실 것 같아요 음식도 맛있습니다 그런데 한 여름이라 동물들이 더울 것 같더라고요',
 '어른들 모시고 갔는데 밑반찬과 갈비찜이 맛있었어요 황태구이는 기름져서 별로 였지만 주차가 편해서 다른 메뉴들도 먹으러 더 방문해보려 합니다',
 '맛있어요 식사가 정성을 많이들인거 같아요',
 '기대이상이였습니다 자반구이정식 꼭 추천드립니다 매운갈비정식도 맛있었어요 막 매운거라 생각했는데 전혀맵지않습니다 애기들 꺼 돈까스는 그냥 쏘쏘']

In [342]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
train_encodings[:1]

[Encoding(num_tokens=146, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]

In [343]:
# UNK 토큰으로 1인 값 0으로 치환 ??
import numpy as np
input_ids = np.array(train_encodings['input_ids'])
for i in range(input_ids.shape[0]):
    input_ids[i][input_ids[i] == 1] = 0

In [344]:
encoded_labels

array([[0, 1, 0, 1, 0],
       [1, 1, 1, 1, 0],
       [0, 1, 0, 0, 1],
       ...,
       [0, 1, 1, 0, 1],
       [0, 1, 0, 0, 1],
       [0, 1, 0, 0, 1]])

In [345]:
label_classes # 총 5개의 라벨 종류

array(['가격', '맛', '서비스', '양', '장소'], dtype=object)

In [346]:
# 원핫인코딩된 레이블 확인
print("원핫인코딩된 레이블의 형태:", encoded_labels.shape)
print("첫 5개의 행의 원핫 인코딩된 레이블:", encoded_labels[:5])
print("레이블 클래스:", label_classes)

# 토큰화 확인
print("리뷰 개수:", len(train_texts))
print("원본 텍스트 예시:", train_texts[0])
print("토큰화된 input_ids 예시:", train_encodings['input_ids'][0])
print("토큰화된 attention_mask 예시:", train_encodings['attention_mask'][0])

# input_ids 길이 확인
print("첫 번째 input_ids의 길이:", len(train_encodings['input_ids'][0]))


원핫인코딩된 레이블의 형태: (17342, 5)
첫 5개의 행의 원핫 인코딩된 레이블: [[0 1 0 1 0]
 [1 1 1 1 0]
 [0 1 0 0 1]
 [0 1 0 0 1]
 [0 0 1 0 1]]
레이블 클래스: ['가격' '맛' '서비스' '양' '장소']
리뷰 개수: 17342
원본 텍스트 예시: 마드레생산구이정식 맛있고 반찬 많아서  외식메뉴로 좋아요
토큰화된 input_ids 예시: [2, 8758, 1105, 1339, 1051, 24210, 14479, 6700, 1133, 11023, 398, 1223, 1107, 5567, 1499, 1531, 1092, 6809, 1296, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
토큰화된 attention_mask 예시: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

- 데이터셋

In [347]:
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertForSequenceClassification, AdamW
import torch.nn as nn

# DataLoader 설정
train_dataset = TensorDataset(
    torch.tensor(train_encodings['input_ids'], dtype=torch.long),
    torch.tensor(train_encodings['attention_mask'], dtype=torch.long),
    torch.tensor(encoded_labels, dtype=torch.float)
)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# 모델 불러오기
num_labels = encoded_labels.shape[1]
model = BertForSequenceClassification.from_pretrained("bongsoo/bert-base-kor-v1", num_labels=num_labels)
model = model.to(device)

# 손실 함수와 옵티마이저 설정
criterion = nn.BCEWithLogitsLoss()

# 수정된 코드
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bongsoo/bert-base-kor-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [348]:
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import KFold

# 임의의 데이터 생성 (input_ids, attention_mask, labels)
num_samples = 1000
num_classes = 5
input_ids = np.random.randint(0, 100, size=(num_samples, 512))
attention_mask = np.random.randint(0, 2, size=(num_samples, 512))
labels = np.random.randint(0, 2, size=(num_samples, num_classes))  #샘플 개수만큼, 이진분류(0 or 1), 다섯개의 라벨이 생성됨
# 원하는 클래스에 1을, 나머지 클래스에 0을 할당/ 각 클래스마다 이진 라벨을 가지도록 데이터를 생성(?)

- 모델 정의

In [349]:
# 모델 정의
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

# class SimpleModel(nn.Module):
#     def __init__(self):
#         super(SimpleModel, self).__init__()
#         self.fc1 = nn.Linear(512, 128)
#         self.fc2 = nn.Linear(128, num_classes)
#         self.dropout = nn.Dropout(0.1)  # 추가: 드롭아웃 레이어

#     def forward(self, input_ids, attention_mask):
#         x = self.fc1(input_ids)
#         x = self.dropout(x)  # 추가: 드롭아웃 적용
#         x = self.fc2(x)
#         return x


In [350]:
# 모델, optimizer, loss 초기화
model = SimpleModel()
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()  #이진 교차 엔트로피 손실을 사용하여 다중 라벨 분류를 지원

# K-Fold와 threshold 설정
kf = KFold(n_splits=5)
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [351]:
# 교차 검증 및 threshold 테스트
for threshold in thresholds:
    f1_scores = []
    accuracy_scores = [] #정확도


    for train_idx, test_idx in kf.split(input_ids):
        train_dataset = TensorDataset(
            torch.tensor(input_ids[train_idx], dtype=torch.float),
            torch.tensor(attention_mask[train_idx], dtype=torch.float),
            torch.tensor(labels[train_idx], dtype=torch.float)

        )
        train_loader = DataLoader(train_dataset, batch_size=32)

        test_dataset = TensorDataset(
            torch.tensor(input_ids[test_idx], dtype=torch.float),
            torch.tensor(attention_mask[test_idx], dtype=torch.float),
            torch.tensor(labels[test_idx], dtype=torch.float)
        )
        test_loader = DataLoader(test_dataset, batch_size=32)

        # 학습
        model.train()
        for batch in train_loader:
            batch_input_ids, batch_attention_mask, batch_labels = batch
            optimizer.zero_grad()
            outputs = model(batch_input_ids)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()

        # 평가
        model.eval()
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for batch in test_loader:
                batch_input_ids, batch_attention_mask, batch_labels = batch
                outputs = model(batch_input_ids)
                all_preds.extend(torch.sigmoid(outputs).cpu().numpy())
                all_labels.extend(batch_labels.cpu().numpy())

        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)

        # 예측 라벨 결정
        predicted_labels = (all_preds > threshold).astype(int)

        # F1 Score 계산
        f1 = f1_score(all_labels, predicted_labels, average='micro')
        f1_scores.append(f1)

        #accuracy score
        accuracy = accuracy_score(all_labels, predicted_labels)
        accuracy_scores.append(accuracy)


    # 결과 출력
    avg_f1 = np.mean(f1_scores)
    avg_accuracy = np.mean(accuracy_scores)

    print(f"Threshold: {threshold}, Avg F1 Score: {avg_f1}, Accuracy Score : {avg_accuracy}")



Threshold: 0.3, Avg F1 Score: 0.6112622309863112, Accuracy Score : 0.063
Threshold: 0.4, Avg F1 Score: 0.6800841957363115, Accuracy Score : 0.129
Threshold: 0.5, Avg F1 Score: 0.6993535195940027, Accuracy Score : 0.126
Threshold: 0.6, Avg F1 Score: 0.705362325193767, Accuracy Score : 0.14
Threshold: 0.7, Avg F1 Score: 0.6888080340937078, Accuracy Score : 0.18699999999999997
Threshold: 0.8, Avg F1 Score: 0.6436039058381292, Accuracy Score : 0.189
Threshold: 0.9, Avg F1 Score: 0.5039680634029263, Accuracy Score : 0.135


In [358]:
print(all_preds)
print(all_labels)

[[8.76522422e-01 1.12902917e-01 9.94329154e-01 6.93718076e-01
  9.98248100e-01]
 [6.28654957e-01 5.73925614e-01 7.89791822e-01 7.32827842e-01
  9.76559818e-01]
 [4.62747902e-01 4.72776413e-01 9.64862347e-01 1.91054285e-01
  9.87323582e-01]
 [3.38945328e-03 5.00945985e-01 8.53666604e-01 2.24680975e-01
  1.20345324e-01]
 [6.43552169e-02 9.04741347e-01 1.57980189e-01 6.79602981e-01
  8.85351241e-01]
 [3.43676120e-01 6.52092516e-01 9.43563700e-01 7.92010605e-01
  9.26007748e-01]
 [1.55653372e-01 2.71269027e-02 9.83825386e-01 9.81085539e-01
  8.75180364e-02]
 [7.64832437e-01 6.95871353e-01 1.99581012e-02 3.12121093e-01
  9.69213188e-01]
 [9.98477280e-01 4.81715649e-02 2.79435515e-01 6.10907257e-01
  8.85361016e-01]
 [2.41371647e-01 7.95754313e-01 4.67744172e-01 5.00647008e-01
  1.11349426e-01]
 [9.74774241e-01 1.01550110e-01 2.90217221e-01 1.99619785e-01
  9.93859112e-01]
 [8.58014524e-01 8.10224861e-02 7.63484299e-01 7.84588873e-01
  2.97329545e-01]
 [8.32783878e-01 3.02575398e-02 4.286435

In [352]:
yogi = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi.csv")

In [353]:
yogi.head()

,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글
0,맛있게 잘먹었습니다. 번창하세요^^,5,5,5,5.0,이쁜 리뷰 감사합니다❤❤❤❤ \n어디에서 이런 가격대비 양 빠른 배달 어디있을까요?...
1,배달해서 먹어도 맛있어요^^♡,5,5,5,5.0,이쁜 리뷰 감사합니다❤❤❤❤ \n고객님께서 저희 알촌을 찾아주셔서 너무 기쁩니다 ㅎ...
2,배달 무척 빠르네요! 맛있게 잘 먹었습니다,4,4,4,5.0,이쁜 리뷰 감사합니다❤❤❤❤ \n고객님 오늘도 이쁜리뷰 감사합니다^_^\n싸다!!...
3,맛있게 먹었습니다,5,5,5,5.0,이쁜 리뷰 감사합니다❤❤❤❤\n가성비 1등 알촌 충북대점을 찾아주셔서 감사합니다^_...
4,맛나게 잘먹었습니다,5,5,5,5.0,이쁜 리뷰 감사합니다❤❤❤❤\n오늘도 저희 알촌을 찾아주셔서 감사드립니다.\n고객님...


In [354]:
new_reviews= yogi["고객리뷰"]

In [355]:
new_reviews

0                                     맛있게 잘먹었습니다. 번창하세요^^
1                                        배달해서 먹어도 맛있어요^^♡
2                                배달 무척 빠르네요!  맛있게 잘 먹었습니다
3                                               맛있게 먹었습니다
4                                              맛나게 잘먹었습니다
                              ...                        
3960                                         맛있게 잘 먹었어요^^
3961           요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요… 그래도 잘 먹었습니다
3962                                 배달 빨라요 맛있게 잘 먹었습니당 !
3963                                          잘 먹었어요~~~~~
3964    #식후감 자주 시키는데 항상 요청사항도 잘들어주시고 맛있고 빨라요 앞으로도 자주 시킬게요
Name: 고객리뷰, Length: 3965, dtype: object

In [356]:
from tqdm.auto import tqdm

def batch_predict(model, input_ids, attention_mask, batch_size):
    model.eval()
    num_batches = len(input_ids) // batch_size + int(len(input_ids) % batch_size != 0)
    probabilities = []
    for batch_num in tqdm(range(num_batches), desc="Predicting"):
        batch_start = batch_num * batch_size
        batch_end = (batch_num + 1) * batch_size
        batch_input_ids = input_ids[batch_start:batch_end].to(device)
        batch_attention_mask = attention_mask[batch_start:batch_end].to(device)
        with torch.no_grad():
            outputs = model(batch_input_ids, attention_mask=batch_attention_mask)
            logits = outputs.logits
            batch_probabilities = torch.sigmoid(logits).cpu()
            probabilities.append(batch_probabilities)
    return torch.cat(probabilities, dim=0)

In [357]:
# 새로운 리뷰 데이터 토큰화
new_reviews_list = new_reviews.tolist()
new_encodings = tokenizer(new_reviews_list, truncation=True, padding=True, max_length=256)

input_ids = torch.tensor(new_encodings['input_ids']).to(device)
attention_mask = torch.tensor(new_encodings['attention_mask']).to(device)

# 평가 모드로 설정
model.eval()

# 예측 수행 (batch_predict 함수 사용)
probabilities = batch_predict(model, input_ids, attention_mask, batch_size=2)

# 예측된 라벨의 확률을 기반으로 실제 키워드 라벨을 결정
threshold = 0.5
predicted_indices = (probabilities > threshold).int()

# MultiLabelBinarizer를 이용하여 예측된 인덱스를 라벨로 변환
predicted_labels = mlb.inverse_transform(predicted_indices.cpu().numpy())

for i, (review, labels) in enumerate(zip(new_reviews, predicted_labels)):
    print(f"리뷰 {i+1}: {review}")
    print(f"예측 라벨: {', '.join(labels)}")
    print()


Predicting:   0%|          | 0/1983 [00:00<?, ?it/s]

TypeError: ignored

In [ ]:
# 새로운 리뷰 데이터 토큰화
new_reviews_list = new_reviews.tolist()
new_encodings = tokenizer(new_reviews_list, truncation=True, padding=True, max_length=256)

input_ids = torch.tensor(new_encodings['input_ids'])
attention_mask = torch.tensor(new_encodings['attention_mask'])

# 평가 모드로 설정
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 예측 수행 (batch_predict 함수 사용)
probabilities = batch_predict(model, input_ids, attention_mask,batch_size=2)

# 예측된 라벨의 확률을 기반으로 실제 키워드 라벨을 결정
threshold = 0.5
predicted_indices = (probabilities > threshold).int()
predicted_labels = mlb.inverse_transform(predicted_indices.numpy())

for i, (review, labels) in enumerate(zip(new_reviews, predicted_labels)):
    print(f"리뷰: {review}")
    print(f"예측 라벨: {labels}")

In [ ]:
one_star = yogi[yogi["별점"] ==5 ]

In [ ]:
one_star["고객리뷰"]

In [ ]:
new_reviews_list = one_star["고객리뷰"]

In [ ]:

# 새로운 리뷰 데이터 토큰화
new_reviews_list = one_star["고객리뷰"].tolist()
new_encodings = tokenizer(new_reviews_list, truncation=True, padding=True, max_length=256)

input_ids = torch.tensor(new_encodings['input_ids'])
attention_mask = torch.tensor(new_encodings['attention_mask'])

# 모델을 평가 모드로 설정
model.eval()

# 모델과 입력 데이터를 동일한 디바이스로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 예측 수행 (batch_predict 함수 사용)
probabilities = batch_predict(model, input_ids, attention_mask, batch_size=1)

# 예측된 라벨의 확률을 기반으로 실제 키워드 라벨을 결정
threshold = 0.5
predicted_indices = (probabilities > threshold).int()
predicted_labels = mlb.inverse_transform(predicted_indices.numpy())

# 결과 출력
for i, (review, labels) in enumerate(zip(new_reviews, predicted_labels)):
    print(f"리뷰: {review}")
    print(f"예측 라벨: {labels}")

# ...


In [ ]:
# 예측된 라벨의 확률을 기반으로 실제 키워드 라벨을 결정
threshold = 0.4
predicted_indices = (probabilities > threshold).int()
predicted_labels = mlb.inverse_transform(predicted_indices.numpy())

# 결과 출력
for i, (review, labels) in enumerate(zip(new_reviews, predicted_labels)):
    print(f"리뷰: {review}")
    print(f"예측 라벨: {labels}")


In [ ]:

# # 새로운 리뷰 데이터 토큰화
# new_reviews_list = one_star["고객리뷰"].tolist()
# new_encodings = tokenizer(new_reviews_list, truncation=True, padding=True, max_length=512)

# input_ids = torch.tensor(new_encodings['input_ids'])
# attention_mask = torch.tensor(new_encodings['attention_mask'])

# # 모델을 평가 모드로 설정
# model.eval()

# # 모델과 입력 데이터를 동일한 디바이스로 이동
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

# # 예측 수행 (batch_predict 함수 사용)
# probabilities = batch_predict(model, input_ids, attention_mask, batch_size=1)

# # 예측된 라벨의 확률을 기반으로 실제 키워드 라벨을 결정
# threshold = 0.5
# predicted_indices = (probabilities > threshold).int()
# predicted_labels = mlb.inverse_transform(predicted_indices.numpy())

# # 결과 출력
# for i, (review, labels) in enumerate(zip(new_reviews_list, predicted_labels)):
#     print(f"리뷰: {review}")
#     print(f"예측 라벨: {labels}")

# # ...


In [ ]:
# from skorch import NeuralNetBinaryClassifier
# import torch

# class BertClassifier(NeuralNetBinaryClassifier):
#     def __init__(self, *args, **kwargs):
#         super(BertClassifier, self).__init__(*args, **kwargs)

#     def initialize(self):
#         self.module_ = self.module(
#             num_labels=len(label_classes),
#         )
#         return self


# 라벨 예측 점수 출력
- ex. 맛 0.7, 가격 0.2, 양 0.1 등
- 다중분류 태스크에서 확률은 이진분류로 나타남 (라벨1에 속할 확률 0.9, 라벨 2에 속할 확률 0.7 등)

In [359]:
import torch
import torch.nn.functional as F

# 모델의 출력 결과 (예시)
model_output = torch.tensor([2.41371647e-01, 7.95754313e-01, 4.67744172e-01, 5.00647008e-01, 1.11349426e-01])

# 소프트맥스 함수를 적용하여 확률 분포로 변환
probabilities = F.softmax(model_output, dim=0)

# 각 라벨이 속할 확률 출력
for label, probability in enumerate(probabilities):
    print(f"라벨 {label}: {probability.item()}")

# 가장 높은 확률을 가진 라벨 출력 (예측된 라벨)
predicted_label = torch.argmax(probabilities).item()
print(f"예측된 라벨: {predicted_label}")


라벨 0: 0.16210147738456726
라벨 1: 0.28219693899154663
라벨 2: 0.2032821923494339
라벨 3: 0.21008199453353882
라벨 4: 0.1423373967409134
예측된 라벨: 1


# ----------참고(작업중)

#합성 데이터 생성(예시)

In [ ]:
# word_mapping = {
#     '맛있어요': '맛없어요',
#     '좋아요': '나빠요',
#     '친절해요': '불친절해요',
#     '편해요': '불편해요',
#     '멋져요': '별로에요',
#     '깔끔해요': '지저분해요',
#     '많아요': '적어요'
# }

# def generate_reverse_reviews(review, word_mapping):
#     reverse_review = review
#     reverse_keywords = []

#     for positive_word, negative_word in word_mapping.items():
#         if positive_word in reverse_review:
#             reverse_review = reverse_review.replace(positive_word, negative_word)
#             reverse_keywords.append(negative_word)

#     return reverse_review, ','.join(reverse_keywords)


In [ ]:
# test_review = "음식이 맛있어요, 서비스도 좋아요, 분위기가 멋져요"
# generate_reverse_reviews(test_review, word_mapping)

In [ ]:
# reverse_rows = []

# # 반전 리뷰 생성
# for index, row in df.iterrows():
#     review = row['review']
#     if not isinstance(review, str):  # 리뷰 문자열 x -> 스킵합니다.
#         continue

#     reverse_review, reverse_keywords = generate_reverse_reviews(review, word_mapping)
#     if reverse_keywords:
#         reverse_rows.append({
#             'review': reverse_review,
#             'keyword': reverse_keywords
#         })


In [ ]:
# reverse_df = pd.DataFrame(reverse_rows)

In [ ]:
# reverse_df.head(20)

# 토큰화

In [ ]:
df

In [ ]:
df['keyword'] = df['keyword'].apply(lambda x: [k.strip() for k in x.split(',')])
df

In [ ]:
df

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertForSequenceClassification, AdamW
import torch.nn as nn

# DataLoader 설정
train_dataset = TensorDataset(
    torch.tensor(train_encodings['input_ids'], dtype=torch.long),
    torch.tensor(train_encodings['attention_mask'], dtype=torch.long),
    torch.tensor(encoded_labels, dtype=torch.float)
)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# 모델 불러오기
num_labels = encoded_labels.shape[1]
model = BertForSequenceClassification.from_pretrained("bongsoo/bert-base-kor-v1", num_labels=num_labels)
model = model.to(device)

# 손실 함수와 옵티마이저 설정
criterion = nn.BCEWithLogitsLoss()

# 수정된 코드
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)

# 데이터셋 및 데이터 로더 설정

# 모델 설정

In [ ]:
print(f"라벨 번호: {num_labels}")
print(f"Dataloader 첫 배치: {next(iter(train_loader))}")
print(f"Model device: {next(model.parameters()).device}")


# 모델학습

In [ ]:
from tqdm import tqdm
import time
import os
from transformers import AdamW, get_linear_schedule_with_warmup
import torch

epochs = 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)

total_steps = len(train_loader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

save_path = "/content/drive/MyDrive/"

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")

    model.train()

    epoch_start_time = time.time()
    total_loss = 0

    for i, batch in tqdm(enumerate(train_loader), total=len(train_loader)):
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        outputs = model(input_ids, attention_mask=attention_mask)  # labels=labels 를 제거
        logits = outputs.logits  # 로짓 가져오기

        loss = criterion(logits, labels.type_as(logits))  # BCEWithLogitsLoss 사용

        total_loss += loss.item()

        loss.backward()


        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 파라미터 업데이트
        optimizer.step()

        # learning rate 조정
        scheduler.step()

        # 그라디언트 초기화
        optimizer.zero_grad()

    avg_train_loss = total_loss / len(train_loader)

    epoch_end_time = time.time()
    epoch_time_elapsed = epoch_end_time - epoch_start_time

    print(f"  평균 학습 loss: {avg_train_loss}")
    print(f"  에폭 토크 학습: {epoch_time_elapsed}s")

    # 저장
    model_save_path = os.path.join(save_path, f"model_epoch_{epoch+1}")
    tokenizer_save_path = os.path.join(save_path, f"tokenizer_epoch_{epoch+1}")
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(tokenizer_save_path)

    print(f"  에폭 저장 {epoch+1}")

print("완료")


In [ ]:
from sklearn.metrics import f1_score, accuracy_score
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertForSequenceClassification
import torch.nn as nn

def permutation_train_test_split(input_ids, attention_mask, target, test_size=0.2, shuffle=True, random_state=SEED):  # SEED 값
    test_num = int(input_ids.shape[0] * test_size)
    train_num = input_ids.shape[0] - test_num

    if shuffle:
        np.random.seed(random_state)
        shuffled = np.random.permutation(input_ids.shape[0])

        input_ids = input_ids[shuffled,:]
        attention_mask = attention_mask[shuffled,:]
        target = target[shuffled,:]

        train_input_ids = input_ids[:train_num]
        train_attention_mask = attention_mask[:train_num]

        test_input_ids = input_ids[train_num:]
        test_attention_mask = attention_mask[train_num:]

        train_target = target[:train_num]
        test_target = target[train_num:]

    else:
        train_input_ids = input_ids[:train_num]
        train_attention_mask = attention_mask[:train_num]

        test_input_ids = input_ids[train_num:]
        test_attention_mask = attention_mask[train_num:]

        train_target = target[:train_num]
        test_target = target[train_num:]

    return train_input_ids, train_attention_mask, test_input_ids, test_attention_mask, train_target, test_target



In [ ]:
# 훈련 데이터와 테스트 데이터 나누기
train_input_ids, train_attention_mask, test_input_ids, test_attention_mask, train_target, test_target = permutation_train_test_split(
    np.array(train_encodings['input_ids']),
    np.array(train_encodings['attention_mask']),
    np.array(encoded_labels),
    test_size=0.2,
    shuffle=True,
    random_state= SEED
)



# 테스트 데이터를 DataLoader에 적용
test_dataset = TensorDataset(
    torch.tensor(test_input_ids, dtype=torch.long),
    torch.tensor(test_attention_mask, dtype=torch.long),
    torch.tensor(test_target, dtype=torch.float)
)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# 모델 평가
model.eval()
test_loss = 0
all_preds = []
all_labels = []

for batch in test_loader:
    input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = criterion(logits, labels.type_as(logits))

    test_loss += loss.item()
    preds = torch.sigmoid(logits)
    all_preds.extend(preds.cpu().numpy())
    all_labels.extend(labels.cpu().numpy())

# 평가 지표 계산
test_loss /= len(test_loader)
predicted_labels = np.round(np.array(all_preds))
accuracy = accuracy_score(np.array(all_labels), predicted_labels)

# 1번: Micro-average F1 Score
f1_micro = f1_score(np.array(all_labels), predicted_labels, average='micro')

# 2번: Macro-average F1 Score
f1_macro = f1_score(np.array(all_labels), predicted_labels, average='macro')

# 3번: Weighted-average F1 Score
f1_weighted = f1_score(np.array(all_labels), predicted_labels, average='weighted')

# 4번: Sample-average F1 Score
f1_samples = f1_score(np.array(all_labels), predicted_labels, average='samples')


print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {accuracy}")
print(f"F1 Score (Micro): {f1_micro}") # 모든 라벨 동등하게 취급(빈도가 낮은 클래스는 높은 클래스에 의해 영향 크게받음)
print(f"F1 Score (Macro): {f1_macro}") #라벨 간 상관관계가 높지 않을 때(모든 라벨 동등하게 취급, 드물게 나타나는 라벨 고려)
print(f"F1 Score (Weighted): {f1_weighted}") #라벨이 불균형할 때 라벨의 중요도 고려(라벨의 빈도수 가중평균)
print(f"F1 Score (Samples): {f1_samples}")  # 각 샘플에 대한 라벨 예측의 정확도를 고려



In [ ]:
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold

# 임의의 데이터 생성 (input_ids, attention_mask, labels)
num_samples = 1000
num_classes = 5
input_ids = np.random.randint(0, 100, size=(num_samples, 512))
attention_mask = np.random.randint(0, 2, size=(num_samples, 512))
labels = np.random.randint(0, 2, size=(num_samples, num_classes))

# 모델 정의
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

# 모델, optimizer, loss 초기화
model = SimpleModel()
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

# K-Fold와 threshold 설정
kf = KFold(n_splits=5)
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]

# 교차 검증 및 threshold 테스트
for threshold in thresholds:
    f1_scores = []

    for train_idx, test_idx in kf.split(input_ids):
        train_dataset = TensorDataset(
            torch.tensor(input_ids[train_idx], dtype=torch.float),
            torch.tensor(attention_mask[train_idx], dtype=torch.float),
            torch.tensor(labels[train_idx], dtype=torch.float)
        )
        train_loader = DataLoader(train_dataset, batch_size=32)

        test_dataset = TensorDataset(
            torch.tensor(input_ids[test_idx], dtype=torch.float),
            torch.tensor(attention_mask[test_idx], dtype=torch.float),
            torch.tensor(labels[test_idx], dtype=torch.float)
        )
        test_loader = DataLoader(test_dataset, batch_size=32)

        # 학습
        model.train()
        for batch in train_loader:
            batch_input_ids, batch_attention_mask, batch_labels = batch
            optimizer.zero_grad()
            outputs = model(batch_input_ids)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()

        # 평가
        model.eval()
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for batch in test_loader:
                batch_input_ids, batch_attention_mask, batch_labels = batch
                outputs = model(batch_input_ids)
                all_preds.extend(torch.sigmoid(outputs).cpu().numpy())
                all_labels.extend(batch_labels.cpu().numpy())

        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)

        # 예측 라벨 결정
        predicted_labels = (all_preds > threshold).astype(int)

        # F1 Score 계산
        f1 = f1_score(all_labels, predicted_labels, average='micro')
        f1_scores.append(f1)

    # 결과 출력
    avg_f1 = np.mean(f1_scores)
    print(f"Threshold: {threshold}, Avg F1 Score: {avg_f1}")


In [ ]:
yogi = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi.csv")

In [ ]:
yogi.head()

In [ ]:
new_reviews= yogi["고객리뷰"]

In [ ]:
new_reviews

In [ ]:
from tqdm.auto import tqdm

def batch_predict(model, input_ids, attention_mask, batch_size):
    model.eval()
    num_batches = len(input_ids) // batch_size + int(len(input_ids) % batch_size != 0)
    probabilities = []
    for batch_num in tqdm(range(num_batches), desc="Predicting"):
        batch_start = batch_num * batch_size
        batch_end = (batch_num + 1) * batch_size
        batch_input_ids = input_ids[batch_start:batch_end].to(device)
        batch_attention_mask = attention_mask[batch_start:batch_end].to(device)
        with torch.no_grad():
            outputs = model(batch_input_ids, attention_mask=batch_attention_mask)
            logits = outputs.logits
            batch_probabilities = torch.sigmoid(logits).cpu()
            probabilities.append(batch_probabilities)
    return torch.cat(probabilities, dim=0)

In [ ]:
# 새로운 리뷰 데이터 토큰화
new_reviews_list = new_reviews.tolist()
new_encodings = tokenizer(new_reviews_list, truncation=True, padding=True, max_length=256)

input_ids = torch.tensor(new_encodings['input_ids'])
attention_mask = torch.tensor(new_encodings['attention_mask'])

# 평가 모드로 설정
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 예측 수행 (batch_predict 함수 사용)
probabilities = batch_predict(model, input_ids, attention_mask, batch_size=1)

# 예측된 라벨의 확률을 기반으로 실제 키워드 라벨을 결정
threshold = 0.5
predicted_indices = (probabilities > threshold).int()
predicted_labels = mlb.inverse_transform(predicted_indices.numpy())

for i, (review, labels) in enumerate(zip(new_reviews, predicted_labels)):
    print(f"리뷰: {review}")
    print(f"예측 라벨: {labels}")

In [ ]:
one_star = yogi[yogi["별점"] ==1 ]

In [ ]:
one_star["고객리뷰"]

In [ ]:
new_reviews_list = one_star["고객리뷰"]

In [ ]:

# 새로운 리뷰 데이터 토큰화
new_reviews_list = one_star["고객리뷰"].tolist()
new_encodings = tokenizer(new_reviews_list, truncation=True, padding=True, max_length=256)

input_ids = torch.tensor(new_encodings['input_ids'])
attention_mask = torch.tensor(new_encodings['attention_mask'])

# 모델을 평가 모드로 설정
model.eval()

# 모델과 입력 데이터를 동일한 디바이스로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 예측 수행 (batch_predict 함수 사용)
probabilities = batch_predict(model, input_ids, attention_mask, batch_size=1)

# 예측된 라벨의 확률을 기반으로 실제 키워드 라벨을 결정
threshold = 0.5
predicted_indices = (probabilities > threshold).int()
predicted_labels = mlb.inverse_transform(predicted_indices.numpy())

# 결과 출력
for i, (review, labels) in enumerate(zip(new_reviews, predicted_labels)):
    print(f"리뷰: {review}")
    print(f"예측 라벨: {labels}")

# ...


In [ ]:
# 예측된 라벨의 확률을 기반으로 실제 키워드 라벨을 결정
threshold = 0.4
predicted_indices = (probabilities > threshold).int()
predicted_labels = mlb.inverse_transform(predicted_indices.numpy())

# 결과 출력
for i, (review, labels) in enumerate(zip(new_reviews, predicted_labels)):
    print(f"리뷰: {review}")
    print(f"예측 라벨: {labels}")


In [ ]:

# # 새로운 리뷰 데이터 토큰화
# new_reviews_list = one_star["고객리뷰"].tolist()
# new_encodings = tokenizer(new_reviews_list, truncation=True, padding=True, max_length=512)

# input_ids = torch.tensor(new_encodings['input_ids'])
# attention_mask = torch.tensor(new_encodings['attention_mask'])

# # 모델을 평가 모드로 설정
# model.eval()

# # 모델과 입력 데이터를 동일한 디바이스로 이동
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

# # 예측 수행 (batch_predict 함수 사용)
# probabilities = batch_predict(model, input_ids, attention_mask, batch_size=1)

# # 예측된 라벨의 확률을 기반으로 실제 키워드 라벨을 결정
# threshold = 0.5
# predicted_indices = (probabilities > threshold).int()
# predicted_labels = mlb.inverse_transform(predicted_indices.numpy())

# # 결과 출력
# for i, (review, labels) in enumerate(zip(new_reviews_list, predicted_labels)):
#     print(f"리뷰: {review}")
#     print(f"예측 라벨: {labels}")

# # ...


In [ ]:
# from skorch import NeuralNetBinaryClassifier
# import torch

# class BertClassifier(NeuralNetBinaryClassifier):
#     def __init__(self, *args, **kwargs):
#         super(BertClassifier, self).__init__(*args, **kwargs)

#     def initialize(self):
#         self.module_ = self.module(
#             num_labels=len(label_classes),
#         )
#         return self


In [ ]:
# from sklearn.metrics import f1_score
# from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
# import torch
# from torch.utils.data import DataLoader, TensorDataset
# import numpy as np
# import time
# from torch.optim import AdamW
# from tqdm import tqdm  # tqdm 추가

# def train_and_evaluate_model(train_loader, test_loader, num_labels, device, epochs=1, learning_rate=2e-5, batch_size=16):
#     # 모델 초기화
#     model = BertForSequenceClassification.from_pretrained("bongsoo/bert-base-kor-v1", num_labels=num_labels)
#     model = model.to(device)

#     optimizer = AdamW(model.parameters(), lr=learning_rate)

#     # 총 훈련 스텝 계산
#     total_steps = len(train_loader) * epochs

#     # 스케줄러 설정
#     scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

#     # 손실 함수 설정
#     criterion = torch.nn.BCEWithLogitsLoss()

#     # 훈련 시작
#     for epoch in range(epochs):
#         model.train()
#         total_loss = 0
#         for i, batch in tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Training epoch {epoch+1}"):
#             input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)
#             outputs = model(input_ids, attention_mask=attention_mask)
#             loss = criterion(outputs.logits, labels.type_as(outputs.logits))
#             total_loss += loss.item()
#             loss.backward()
#             optimizer.step()
#             scheduler.step()
#             optimizer.zero_grad()

#         # 평가 시작
#         model.eval()
#         all_preds = []
#         all_labels = []
#         for i, batch in tqdm(enumerate(test_loader), total=len(test_loader), desc=f"Evaluating epoch {epoch+1}"):
#             input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)
#             with torch.no_grad():
#                 outputs = model(input_ids, attention_mask=attention_mask)
#                 logits = outputs.logits
#             preds = torch.sigmoid(logits)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())

#         # F1 스코어 계산
#         predicted_labels = np.round(np.array(all_preds))
#         f1 = f1_score(np.array(all_labels), predicted_labels, average='micro')
#         print(f"Epoch {epoch+1}/{epochs} - F1 Score: {f1}")

#     return f1
# # 하이퍼파라미터 그리드 설정
# param_grid = {
#     'epochs': [1, 2],
#     'learning_rate': [2e-5, 3e-5],
#     'batch_size': [16, 32]
# }

# # 최적 F1 스코어 저장 변수 초기화
# best_f1 = 0
# best_params = {}

# # 그리드 서치 시작
# for epochs in param_grid['epochs']:
#     for learning_rate in param_grid['learning_rate']:
#         for batch_size in param_grid['batch_size']:
#             print(f"Training with epochs={epochs}, learning_rate={learning_rate}, batch_size={batch_size}")

#             # DataLoader 설정
#             train_dataset = TensorDataset(
#                 torch.tensor(train_input_ids, dtype=torch.long),
#                 torch.tensor(train_attention_mask, dtype=torch.long),
#                 torch.tensor(train_target, dtype=torch.float)
#             )
#             train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#             test_dataset = TensorDataset(
#                 torch.tensor(test_input_ids, dtype=torch.long),
#                 torch.tensor(test_attention_mask, dtype=torch.long),
#                 torch.tensor(test_target, dtype=torch.float)
#             )
#             test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#             # 모델 훈련 및 평가
#             f1 = train_and_evaluate_model(train_loader, test_loader, num_labels=len(label_classes), device=device, epochs=epochs, learning_rate=learning_rate, batch_size=batch_size)

#             # 최적 F1 스코어 업데이트
#             if f1 > best_f1:
#                 best_f1 = f1
#                 best_params = {'epochs': epochs, 'learning_rate': learning_rate, 'batch_size': batch_size}

# print(f"Best F1 Score: {best_f1}")
# print(f"Best Parameters: {best_params}")
